In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import string
import warnings
import missingno
warnings.filterwarnings('ignore')


import pandas as pd
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import numpy as np
from numpy.random import gamma
from numpy.random import multivariate_normal
from scipy.stats import multivariate_t
from scipy.stats import f
import matplotlib.pyplot as plt



cf.go_offline(connected = True)


import pandas as pd
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import numpy as np
from numpy.random import gamma
from numpy.random import multivariate_normal
from scipy.stats import multivariate_t
from scipy.stats import f
import matplotlib.pyplot as plt
import missingno


cf.go_offline(connected = True)


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
import string
import warnings
import missingno

D:\anaconda\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 전체 프로세스

- 1) 기관의 요청 or 기관이 직접 발주계획
- 2) 입찰 공고
- 3) 예비가격 생성 ( 예비 -> 예정 가격을 만들기 위한 과정 ; 복잡하다)
- 4) PQ 검사 / 설명회
- 5) 투찰 (참여 업체가 값을 적어 냄)
- 6) 개찰 (참여 업체가 투찰한 금액을 염 이때 예정가격과 가장 비슷한 가격을 선택)
- 7) 적격심사 대상 / 협상 대상 인 경우 확인
- 8) 낙찰 완료




> 입찰 :	입찰은 낙찰 희망 가격을 서면으로 제출하는 것을 뜻합니다

> 투찰 : 	투찰은 참가하여 가격을 써 내는 것을 의미합니다

> 개찰 :	개찰은 입찰한 결과를 조사하는것을 의미합니다

> 낙찰 :	낙찰은 경매 따위에서 물건이나 일이 어떤 사람이나 업체에 돌아가도록 결정되는것을 의미

> 조달 :	자금이나 물자 따위를 대어 준다는 뜻을 지칭

> 발주 :	물건을 보내달라고 하는 주문


### 수요기관별 입찰집행실적 집계에서는 낙찰금액이 적혀있는 것이 적었다. 이는 실제로 예산이 사용된 것은 알 수 있으나, 수요에 대해서는 (예산을 사용하기로 예정한 대상)를 알기에는 부적절하다고 생각이 되었다. 그러므로, 수요기관별 입찰집행실적 집계를 보기 보다는, 입찰 공고에 있는 추정금액이 곧 기관의 수요라고 생각하였다. 실제 사용은 입찰집행실적이겠지만, 우리가 보려는 것은 기관의 '예측 수요'이다. 

### 지금 헷갈리는게 전체적인 프로세스가 끝난 뒤의 데이터를 볼 것인지 (입찰 집행 실적 집계) 프로세스가 완전히 다 진행되기 전의 (투찰  / 개찰 되기 전) 데이터를 볼 것인지가 하나의 의문점인데, 프로세스가 다 완전히 다 진행되고 난 다음에 데이터를 본다면 실제 수요에 대한 예측을 할 수 있지만 우리가 목적으로 하는 예정 수요에 대한 예측을 수행하기 어렵다. 우리가 만드려는 모델은 입찰공고를 기관이 딱 내릴 때, 그 것에 대해서 감지를 하는 관리도를 만들려는 것이다. 그리고 In-control 상태의 데이터만을 이용해서 예정 금액의 예측 모델을 만드는 것이다. 

### 그렇다면 명확해진다. 프로세스가 다 진행된 뒤의 금액은 처음에 배정한 금액보다 작을 것이 대부분이다. 그렇다면, 새로운 데이터(새로운 입찰공고를 내렸을 때의 배정금액)은 낙찰가격(프로세스가 다 진행한 후의 금액 데이터)보다는 더 낮을 것이므로(특수 경우 제외하고는 이게 대부분) 예측이나 관리도 구축에 있어서 우리가 원하는 것과 다르게 나올 것이다. 그러므로 입찰 공고를 보는 것이 더 합리적이라고 생각한다.

In [3]:
df = pd.read_csv('물품_입찰공고_내역_2020.csv',encoding ='utf-8',error_bad_lines= False)
df = pd.concat([df,pd.read_csv('물품_입찰공고_내역_2019.csv',encoding ='utf-8',error_bad_lines= False)])
df = pd.concat([df,pd.read_csv('물품_입찰공고_내역_2018.csv',encoding ='utf-8',error_bad_lines= False)])
df_물품_입찰공고_내역 = pd.concat([df,pd.read_csv('물품_입찰공고_내역_2017.csv',encoding ='utf-8',error_bad_lines= False)])

b'Skipping line 114977: expected 55 fields, saw 56\n'


In [4]:
def make_time_series_3(row):
    lst_all = []
    for i in row:
        i = str(i)
        year = i[0:4]
        month = i[4:6]
        date =  i[6:8]

        lst_all.append([pd.to_datetime(year+month+date,format='%Y%m%d')])
    return lst_all

In [5]:
df_물품_입찰공고_내역.columns


df_물품_입찰공고_내역_2 = df_물품_입찰공고_내역[[ '입찰공고번호', '입찰공고차수', '공고게시일자',  '수요기관명', '수요기관코드', '수요기관구분', '품명내용',
       '세부품명내용', '합계수량', '합계금액', '납품기한', '긴급공고여부', '정정공고여부', '취소공고여부', '추정가격',
       '배정예산',  '재입찰허용여부',
       '조달요청번호']].copy()




# In[259]:
wbc= make_time_series_3(df_물품_입찰공고_내역_2['공고게시일자'])

df_물품_입찰공고_내역_2.reset_index(inplace=True)
df_물품_입찰공고_내역_2.loc[:,'공고게시일자'] =wbc

df_물품_입찰공고_내역_2['수요기관코드'] = df_물품_입찰공고_내역_2['수요기관코드'].astype('str')

df_test = df_물품_입찰공고_내역_2[df_물품_입찰공고_내역_2['수요기관코드'] == str(1690000)]
df_test.head(2)

,index,입찰공고번호,입찰공고차수,공고게시일자,수요기관명,수요기관코드,수요기관구분,품명내용,세부품명내용,합계수량,합계금액,납품기한,긴급공고여부,정정공고여부,취소공고여부,추정가격,배정예산,재입찰허용여부,조달요청번호
1305,1305,20200318789,0,2020-03-12,방위사업청,1690000,국가기관,토너[44103103],정품토너[4410310302],1.0,2.192444e+08,계약 후 10일 이내,NaN,N,N,1.993131e+08,2.208254e+08,Y,1220209690.0
34606,34606,20200429508,0,2020-04-21,방위사업청,1690000,국가기관,노트북컴퓨터[43211503],노트북컴퓨터[4321150301],1.0,2.458072e+09,"1차 2020/07/29, 2차 2021/09/29",NaN,N,N,2.234611e+09,2.469999e+09,Y,1220814094.0


In [6]:
df_물품_입찰공고_내역_new = pd.read_csv('물품_입찰공고_내역_2021.csv',encoding ='utf-8',error_bad_lines= False)

df_물품_입찰공고_내역_new.columns


# In[256]:


df_물품_입찰공고_내역_new = df_물품_입찰공고_내역_new[ [ '입찰공고번호', '입찰공고차수', '공고게시일자',  '수요기관명', '수요기관코드', '수요기관구분', '품명내용',
       '세부품명내용', '합계수량', '합계금액', '납품기한', '긴급공고여부', '정정공고여부', '취소공고여부', '추정가격',
       '배정예산',  '재입찰허용여부',
       '조달요청번호']].copy()

df_물품_입찰공고_내역_new['공고게시일자'] = pd.DataFrame(make_time_series_3(df_물품_입찰공고_내역_new['공고게시일자']))
# In[257]:



# In[272]:


df_test_2 = df_물품_입찰공고_내역_new[df_물품_입찰공고_내역_new['수요기관코드'] == str(1690000)]

In [23]:
df = pd.read_csv('공공기관별_업무대상별_실적_2021.csv',encoding ='utf-8',error_bad_lines= False).iloc[[1]]
df = pd.concat([df,pd.read_csv('공공기관별_업무대상별_실적_2020.csv',encoding ='utf-8',error_bad_lines= False).iloc[[1]]])
df = pd.concat([df,pd.read_csv('공공기관별_업무대상별_실적_2019.csv',encoding ='utf-8',error_bad_lines= False).iloc[[1]]])
df = pd.concat([df,pd.read_csv('공공기관별_업무대상별_실적_2018.csv',encoding ='utf-8',error_bad_lines= False).iloc[[1]]])
df_공공기관별_업무대상별_실적 = pd.concat([df,pd.read_csv('공공기관별_업무대상별_실적_2017.csv',encoding ='utf-8',error_bad_lines= False).iloc[[1]]])

In [24]:
df_공공기관별_업무대상별_실적.index = [2021,2020,2019,2018,2017]

In [25]:
df_공공기관별_업무대상별_실적

,기관명,전체건수,전체금액,물품건수,물품금액,공사건수,공사금액,일반용역건수,일반용역금액,기술용역건수,기술용역금액
2021,방위사업청,2471,12588957128187,2418,12562373238070,11,219821317,41,26202808800,1,161260000
2020,방위사업청,2534,10168315450867,2500,10149511443137,1,21886000,32,18332121730,1,450000000
2019,방위사업청,2407,12085901177043,2381,12069876576160,2,130723643,23,15544737240,1,349140000
2018,방위사업청,2400,11731372202662,2375,11718984051552,2,42493700,22,12155418410,1,190239000
2017,방위사업청,2838,9499060220825,2797,9485180233837,4,1152792648,36,12357194340,1,370000000


In [26]:
df_공공기관별_업무대상별_실적.loc[2020:2017,].describe()[['물품금액']]

,물품금액
count,4.000000e+00
mean,1.085589e+13
std,1.237809e+12
min,9.485180e+12
25%,9.983429e+12
50%,1.093425e+13
75%,1.180671e+13
max,1.206988e+13


In [83]:
mean = pd.Series(df_공공기관별_업무대상별_실적.loc[2020:2017,]['물품금액'].agg({2021:'mean'}))
mean

2021    1.085589e+13
Name: 물품금액, dtype: float64

In [84]:
df_공공기관별_업무대상별_실적.loc[2020:2017,][['물품금액']]

,물품금액
2020,10149511443137
2019,12069876576160
2018,11718984051552
2017,9485180233837


In [91]:
df_실적_test = pd.DataFrame(df_공공기관별_업무대상별_실적.loc[2020:2017,]['물품금액'].append(mean))

In [96]:
df_실적_test = df_실적_test / df_실적_test.loc[2021,'물품금액']

In [97]:
df_실적_test

,물품금액
2020,0.934931
2019,1.111828
2018,1.079505
2017,0.873736
2021,1.000000


In [101]:
dt = pd.DatetimeIndex(df_test['공고게시일자'])

df_test['day'] = dt.day
df_test['month'] = dt.month
df_test['year'] = dt.year
df_test['hour'] = dt.hour
df_test['dow'] = dt.dayofweek
df_test['woy'] = dt.weekofyear


dt = pd.DatetimeIndex(df_test_2['공고게시일자'])

df_test_2['day'] = dt.day
df_test_2['month'] = dt.month
df_test_2['year'] = dt.year
df_test_2['hour'] = dt.hour
df_test_2['dow'] = dt.dayofweek
df_test_2['woy'] = dt.weekofyear

In [134]:
df_test_1_grouper = df_test.groupby(pd.Grouper(key = '공고게시일자',freq = '1W')).sum()
df_test_2_grouper = df_test_2.groupby(pd.Grouper(key = '공고게시일자',freq = '1W')).sum()

In [135]:
df_test_1_grouper['실적'] = 0

In [136]:
df_test_1_grouper

,index,입찰공고차수,합계수량,합계금액,추정가격,배정예산,day,month,year,hour,dow,woy,실적
공고게시일자,,,,,,,,,,,,,
2017-01-29,0,0,5.0,170000000.0,154545455.0,1.708266e+08,24,1,2017,0,1,4,0
2017-02-05,6,0,5.0,170000000.0,154545455.0,1.708266e+08,2,2,2017,0,3,5,0
2017-02-12,7,0,5.0,170000000.0,154545455.0,1.708266e+08,10,2,2017,0,4,6,0
2017-02-19,1,0,1.0,443847600.0,403497818.0,4.454314e+08,16,2,2017,0,3,7,0
2017-02-26,2,0,5.0,170000000.0,154545455.0,1.708266e+08,23,2,2017,0,3,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-06,12618210,139,1.0,219910800.0,199918909.0,1.158117e+11,875,1262,216140,0,215,5243,0
2020-12-13,10687813,131,0.0,0.0,0.0,7.513172e+10,838,1080,181800,0,208,4500,0
2020-12-20,6544001,82,0.0,0.0,0.0,1.394417e+10,881,660,111100,0,111,2805,0


In [137]:
df_실적_test.loc[df_실적_test.index == 2017 ,'물품금액']

2017    0.873736
Name: 물품금액, dtype: float64

In [138]:
df_test_1_grouper.loc[df_test_1_grouper.index[0],'실적'] =1 

In [139]:
df_test_1_grouper.index

DatetimeIndex(['2017-01-29', '2017-02-05', '2017-02-12', '2017-02-19',
               '2017-02-26', '2017-03-05', '2017-03-12', '2017-03-19',
               '2017-03-26', '2017-04-02',
               ...
               '2020-11-01', '2020-11-08', '2020-11-15', '2020-11-22',
               '2020-11-29', '2020-12-06', '2020-12-13', '2020-12-20',
               '2020-12-27', '2021-01-03'],
              dtype='datetime64[ns]', name='공고게시일자', length=206, freq='W-SUN')

In [146]:
df_실적_test

,물품금액
2020,0.934931
2019,1.111828
2018,1.079505
2017,0.873736
2021,1.000000


In [140]:
for j,i in enumerate(df_test_1_grouper.index): 
    if i.year == 2017:
        df_test_1_grouper.loc[df_test_1_grouper.index[j],'실적'] = df_실적_test.loc[df_실적_test.index == 2017 ,'물품금액'].values
    elif i.year == 2018:
        df_test_1_grouper.loc[df_test_1_grouper.index[j],'실적'] = df_실적_test.loc[df_실적_test.index == 2018,'물품금액'].values
    elif i.year == 2019:
        df_test_1_grouper.loc[df_test_1_grouper.index[j],'실적'] = df_실적_test.loc[df_실적_test.index == 2019,'물품금액'].values
    elif i.year == 2020:
        df_test_1_grouper.loc[df_test_1_grouper.index[j],'실적'] = df_실적_test.loc[df_실적_test.index == 2020,'물품금액'].values
    elif i.year == 2021:
        df_test_1_grouper.loc[df_test_1_grouper.index[j],'실적'] = df_실적_test.loc[df_실적_test.index == 2021,'물품금액'].values


In [142]:
for j,i in enumerate(df_test_2_grouper.index): 
    if i.year == 2017:
        df_test_2_grouper.loc[df_test_2_grouper.index[j],'실적'] = df_실적_test.loc[df_실적_test.index == 2017 ,'물품금액'].values
    elif i.year == 2018:
        df_test_2_grouper.loc[df_test_2_grouper.index[j],'실적'] = df_실적_test.loc[df_실적_test.index == 2018,'물품금액'].values
    elif i.year == 2019:
        df_test_2_grouper.loc[df_test_2_grouper.index[j],'실적'] = df_실적_test.loc[df_실적_test.index == 2019,'물품금액'].values
    elif i.year == 2020:
        df_test_2_grouper.loc[df_test_2_grouper.index[j],'실적'] = df_실적_test.loc[df_실적_test.index == 2020,'물품금액'].values
    elif i.year == 2021:
        df_test_2_grouper.loc[df_test_2_grouper.index[j],'실적'] = df_실적_test.loc[df_실적_test.index == 2021,'물품금액'].values

In [145]:
df_test_2_grouper[df_test_2_grouper['실적'].isnull()]

,입찰공고차수,합계수량,합계금액,추정가격,배정예산,day,month,year,hour,dow,woy,실적
공고게시일자,,,,,,,,,,,,
2022-01-02,18,0.0,0.0,0.0,2.377210e+09,446,180,30315,0,41,780,NaN
2022-01-09,30,0.0,0.0,0.0,2.735366e+09,107,18,36396,0,53,18,NaN
2022-01-16,44,0.0,0.0,0.0,1.169948e+10,401,32,64704,0,81,64,NaN
2022-01-23,8,0.0,0.0,0.0,4.061218e+08,69,4,8088,0,1,12,NaN


In [154]:
df_test_1_grouper.isnull().sum()

index     0
입찰공고차수    0
합계수량      0
합계금액      0
추정가격      0
배정예산      0
day       0
month     0
year      0
hour      0
dow       0
woy       0
실적        0
dtype: int64

In [155]:
df_test_1_grouper.dropna(subset=['실적'],inplace = True)

In [156]:
df_test_1_grouper.isnull().sum()

index     0
입찰공고차수    0
합계수량      0
합계금액      0
추정가격      0
배정예산      0
day       0
month     0
year      0
hour      0
dow       0
woy       0
실적        0
dtype: int64

In [157]:
df_test_2_grouper.isnull().sum()

입찰공고차수    0
합계수량      0
합계금액      0
추정가격      0
배정예산      0
day       0
month     0
year      0
hour      0
dow       0
woy       0
실적        0
dtype: int64

In [158]:
df_test_2_grouper.dropna(subset=['실적'],inplace = True)

In [159]:
df_test_2_grouper.isnull().sum()

입찰공고차수    0
합계수량      0
합계금액      0
추정가격      0
배정예산      0
day       0
month     0
year      0
hour      0
dow       0
woy       0
실적        0
dtype: int64

In [162]:
ml_columns = [
'day', 'month', 'year', 'hour', 'dow', 'woy','실적','입찰공고차수'
]

In [163]:
X_train = df_test_1_grouper[ml_columns].copy()
y_train = df_test_1_grouper['배정예산'].copy()
X_test = df_test_2_grouper[ml_columns].copy()
y_test = df_test_2_grouper['배정예산'].copy()

In [166]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model_made_1 = model.fit(X_train, y_train)


In [167]:
model_made_1.predict(X_test)

array([4.38279224e+10, 1.42541443e+10, 2.40824456e+10, 7.46054052e+10,
       1.27707786e+11, 5.85368880e+09, 2.09394541e+10, 5.63177804e+10,
       9.71644567e+10, 7.05580972e+10, 6.80103259e+10, 8.83905655e+10,
       9.82628772e+10, 9.40635202e+10, 7.73401097e+10, 1.16953434e+11,
       8.71122676e+10, 1.14001974e+11, 8.45898000e+10, 7.70198650e+10,
       1.03726405e+11, 1.01263092e+11, 8.14532429e+10, 7.41423480e+10,
       8.11501522e+10, 8.12557455e+10, 9.08491061e+10, 8.70301350e+10,
       8.94697387e+10, 8.57674155e+10, 1.04674035e+11, 1.19364742e+11,
       1.42936916e+11, 1.45397275e+11, 4.82709894e+10, 1.02042099e+11,
       1.07889670e+11, 9.74433455e+10, 1.30143801e+11, 1.15237290e+11,
       1.25367738e+11, 1.22553650e+11, 1.04075659e+11, 9.93193705e+10,
       1.05870714e+11, 2.02530965e+11, 1.47867605e+11, 6.64514211e+10])

In [169]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ',score(model_made_1, X_test, y_test))

score = make_scorer(r2_score)

print('r2_score = ',score(model_made_1, X_test, y_test))

mean_squared_log_error =  1.1006352434286615
r2_score =  0.027563938800382992


In [394]:
n_estimators = [200, 500, 750]
max_depth = [4, 5, 6]
min_samples_leaf = [12, 15]
learning_rate = [0.02, 0.05, 0.1]
subsample = [0.6, 0.7, 0.8]

hyperparams = {'n_estimators': n_estimators, 'max_depth': max_depth, 
                    'min_samples_leaf': min_samples_leaf,
                    'learning_rate': learning_rate, 'subsample': subsample
              }


XGB_grid = GridSearchCV(estimator = XGBRegressor(), param_grid = hyperparams, 
                verbose=True, scoring=mean_squared_log_error, cv=5, n_jobs=-1)

XGB_grid.fit(X_train, y_train)
print(XGB_grid.best_params_)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[11:22:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "min_samples_leaf" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


{'learning_rate': 0.02, 'max_depth': 4, 'min_samples_leaf': 12, 'n_estimators': 200, 'subsample': 0.6}


In [395]:
XGB_grid_1 = XGB_grid.best_estimator_
XGB_grid_2 = XGB_grid_1.fit(X_train, y_train)
XGB_grid_2_predict = XGB_grid_2.predict(X_test)

[11:22:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "min_samples_leaf" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [396]:
fig = go.Figure()
fig.add_trace(go.Bar(x =X_train.columns  , y =XGB_grid_2.feature_importances_ ))

fig.update_layout(title = 'XGBoost에 대한 Feature importance')

In [397]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(XGB_grid_2, X_train, y_train))


score = make_scorer(r2_score)

print('r2_score = ', score(XGB_grid_2, X_train, y_train))

mean_squared_log_error =  43.057487256537144
r2_score =  0.8879474401287542


In [398]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(XGB_grid_2, X_test, y_test))


score = make_scorer(r2_score)

print('r2_score = ', score(XGB_grid_2, X_test, y_test))

mean_squared_log_error =  0.7375139895808069
r2_score =  0.1154657863569597


In [399]:
XGB_grid_2_predict_train = XGB_grid_2.predict(X_train)

In [400]:
df1 = pd.DataFrame(XGB_grid_2_predict,index=X_test.index)
df2 = pd.DataFrame(y_test,index = X_test.index)
df1_1 = pd.DataFrame(XGB_grid_2_predict_train,index=X_train.index)
df2_1 = pd.DataFrame(y_train,index = X_train.index)

In [177]:

n_estimators = [800, 1000, 1200]
max_depth = [10, 12, 15]
min_samples_split = [4, 5, 6]
min_samples_leaf = [4, 5, 6]

hyperparams = {'n_estimators': n_estimators, 'max_depth': max_depth, 
               'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}

random_grid = GridSearchCV(estimator = RandomForestRegressor(), param_grid = hyperparams, 
                verbose=True, scoring=mean_squared_log_error, cv=5, n_jobs=-1)

random_grid.fit(X_train, y_train)
print(random_grid.best_params_)


Fitting 5 folds for each of 81 candidates, totalling 405 fits
{'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 800}


In [178]:
random_grid_1= random_grid.best_estimator_
random_grid_2 = random_grid_1.fit(X_train, y_train)
random_grid_2_predict = random_grid_2.predict(X_test)

Index(['day', 'month', 'year', 'hour', 'dow', 'woy', '실적', '입찰공고차수'], dtype='object')

In [199]:
fig = go.Figure()
fig.add_trace(go.Bar(x =X_train.columns  , y =random_grid_1.feature_importances_ ))

fig.update_layout(title = 'RandomForestRegressor에 대한 Feature importance')


In [179]:
df3 = pd.DataFrame(random_grid_2_predict,index=X_test.index)
df3_1 = pd.DataFrame(random_grid_2.predict(X_train),index=X_train.index)


In [183]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(random_grid_2, X_train, y_train))


score = make_scorer(r2_score)

print('r2_score = ', score(random_grid_2, X_train, y_train))

mean_squared_log_error =  27.667707562547324
r2_score =  0.5314713557217025


In [184]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(random_grid_2, X_test, y_test))


score = make_scorer(r2_score)

print('r2_score = ', score(random_grid_2, X_test, y_test))

mean_squared_log_error =  0.7770217224420554
r2_score =  0.0806643561258743


In [479]:
n_estimators = [200, 500, 750]
max_depth = [4, 5, 6]
min_samples_leaf = [12, 15]
learning_rate = [0.02, 0.05, 0.1]
subsample = [0.6, 0.7, 0.8]

hyperparams = {'n_estimators': n_estimators, 'max_depth': max_depth, 
                    'min_samples_leaf': min_samples_leaf,
                    'learning_rate': learning_rate, 'subsample': subsample
              }


LightGB_grid = GridSearchCV(estimator = LGBMRegressor(), param_grid = hyperparams, 
                verbose=True, scoring=mean_squared_log_error, cv=5, n_jobs=-1)

LightGB_grid.fit(X_train, y_train)
print(LightGB_grid.best_params_)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=12. Current value: min_data_in_leaf=12
{'learning_rate': 0.02, 'max_depth': 4, 'min_samples_leaf': 12, 'n_estimators': 200, 'subsample': 0.6}


In [480]:
LightGB_grid_1= LightGB_grid.best_estimator_
LightGB_grid_2 = LightGB_grid_1.fit(X_train, y_train)
LightGB_grid_2_predict = LightGB_grid_2.predict(X_test)

In [481]:
fig = go.Figure()
fig.add_trace(go.Bar(x =X_train.columns  , y =LightGB_grid_2.feature_importances_ ))

fig.update_layout(title = 'LightGBM에 대한 Feature importance')


In [482]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(LightGB_grid_2, X_train, y_train))


score = make_scorer(r2_score)

print('r2_score = ', score(LightGB_grid_2, X_train, y_train))

mean_squared_log_error =  46.173373511547645
r2_score =  0.5816968793149475


In [483]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(LightGB_grid_2, X_test, y_test))


score = make_scorer(r2_score)

print('r2_score = ', score(LightGB_grid_2, X_test, y_test))

mean_squared_log_error =  0.655322514813211
r2_score =  0.09825468391342451


In [419]:
LightGB_grid_2_predict_train = LightGB_grid_2.predict(X_train)
LightGB_grid_2_predict_test = LightGB_grid_2_predict
df4_1 = pd.DataFrame(LightGB_grid_2_predict_train,index=X_train.index)
df4 = pd.DataFrame(LightGB_grid_2_predict_test,index = X_test.index)

In [423]:
# Total
fig = go.Figure()

fig.add_trace(go.Scatter(x = df2.index, y = df2[df2_1.columns[0]],name = 'Test_실제값',marker_color = 'blue'))
fig.add_trace(go.Scatter(x = df2_1.index, y = df2_1[df2_1.columns[0]],name = 'Train_실제값',marker_color = 'blue'))
fig.add_trace(go.Scatter(x = df1.index, y = df1[0], name = 'Test_예측값_XGBOOST',marker_color = 'red'))#,mode='lines+markers'

fig.add_trace(go.Scatter(x = df1_1.index, y = df1_1[0], name = 'Train_예측값_XGBOOST',marker_color = 'red'))


fig.add_trace(go.Scatter(x = df3.index, y = df3[0], name = 'Test_예측값_RandomForest',marker_color = 'green'))

fig.add_trace(go.Scatter(x = df3_1.index, y = df3_1[0], name = 'Train_예측값_RandomForest',marker_color = 'green'))

fig.add_trace(go.Scatter(x = df4.index, y = df4[0], name = 'Test_예측값_LightGB',marker_color = 'orange'))

fig.add_trace(go.Scatter(x = df4_1.index, y = df4_1[0], name = 'Train_예측값_LightGB',marker_color = 'orange'))


fig.add_vline(x = '2021-01-01',line_dash="dash")
fig.add_annotation(
            x="2021-02-20",
            y=10**12,
            text="<b>Test</b>",
            showarrow=False,
            font=dict(
                size=15,
                color = 'black'
                ),
            align="center",
            ax=40,
            ay=-30,

)
fig.add_annotation(
            x="2020-11-15",
            y=10**12,
            text="<b>Train</b>",
            showarrow=False,
            font=dict(
                size=15,
                color = 'black'
                ),
            align="center",
            ax=40,
            ay=-30,

)

fig.update_layout(
    autosize=False,
    width=1600,
    height=800,
    title= '물품 : 방위사업청 대한 수요예측 '
    )

fig.update_xaxes(title = '주 단위 시계열 (2020/01/05 ~ 2021/01/23)')
fig.update_yaxes(title = '주별 예산 총액')

fig.write_html("주별.html")
fig.show()

In [203]:
# import joblib

# joblib.dump(model_made_1, 'Linear_predict.pkl')


# joblib.dump(random_grid_2, 'random_grid.pkl')

# joblib.dump(XGB_grid_2, 'XGB_grid.pkl')

['XGB_grid.pkl']

## 생각했던 것보다 모델의 성능이 좋아지지 않았다. 또한, XGBoost에서와 Random Forest에서 중요하게 보는 변수가 많이 달랐다. 

- 1) 다른 모형 (Extra Trees regression, SVR 
- 2) 그러므로 이를 혼합한 Voting regression을 사용하여 보자.

In [215]:
import pandas as pd
import numpy as np # 각 모델에서 내부적으로 관련 라이브러리 사용 가능

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

from sklearn.neighbors import KNeighborsClassifier             # 1. K-Nearest Neighbor(KNN)
from sklearn.linear_model import LogisticRegression            # 2. Logistic Regression
from sklearn.svm import SVC                                                # 3. SVC
from sklearn.tree import DecisionTreeClassifier                   # 4. Decision Tree
from sklearn.ensemble import RandomForestClassifier       # 5. Random Forest
from sklearn.ensemble import ExtraTreesClassifier  # 6. ExtraTrees
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier # 7. GBM
from sklearn.naive_bayes import GaussianNB                     # 8. GaussianNB
from xgboost import XGBClassifier                                     # 9. XGBoost
from lightgbm import LGBMClassifier                                 # 10. LightGBM
import warnings
from sklearn.svm import SVR  

In [220]:
# 파라미터 그리드 셋팅
hyperparams = {
    'C': [10, 15, 20, 23, 25, 30, 50], 
    'gamma' : [0.001, 0.01, 0.05, 0.06, 0.07, 0.1]
}

# 교차검증
gd=GridSearchCV(
    estimator = SVR(verbose=True), 
    param_grid = hyperparams, 
    verbose=True, 
    cv=5,   # 내부적으로 (Stratified)KFold 사용
    scoring = "neg_mean_squared_log_error", 
    n_jobs=-1
)

# 모델 fiting 및 결과
gb_1 = gd.fit(X_train, y_train)


print(gd.best_params_)

gd_1 = gd.best_estimator_
gd_2 = gd_1.fit(X_train, y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits
[LibSVM]{'C': 50, 'gamma': 0.001}
[LibSVM]

In [221]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(gd_1, X_test, y_test))


score = make_scorer(r2_score)

print('r2_score = ', score(gd_1, X_test, y_test))

mean_squared_log_error =  1.142244863228683
r2_score =  -0.13313119262317397


In [217]:

n_estimators = [800, 1000, 1200]
max_depth = [10, 12, 15]
min_samples_split = [4, 5, 6]
min_samples_leaf = [4, 5, 6]

hyperparams = {'n_estimators': n_estimators, 'max_depth': max_depth, 
               'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}

random_grid_extra = GridSearchCV(estimator = ExtraTreesRegressor(), param_grid = hyperparams, 
                verbose=True, scoring=mean_squared_log_error, cv=5, n_jobs=-1)

random_grid_extra.fit(X_train, y_train)
print(random_grid_extra.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
{'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 800}


In [218]:
random_grid_extra_1= random_grid_extra.best_estimator_
random_grid_extra_2 = random_grid_extra_1.fit(X_train, y_train)
random_grid_extra_2_predict = random_grid_extra_2.predict(X_test)

In [219]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(random_grid_extra_2, X_test, y_test))


score = make_scorer(r2_score)

print('r2_score = ', score(random_grid_extra_2, X_test, y_test))

mean_squared_log_error =  0.7893187087938293
r2_score =  0.036750090526203594



### Bayesian Optimization XGBoost 적용

In [346]:
import numpy as np
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

pbounds = {  
    'learning_rate': (0.01, 0.5),  
    'n_estimators': (100, 1000), 
    'max_depth': (3, 10),
    'min_child_weight': (0, 20),
    'subsample': (0.5, 1.0),  
    'colsample_bytree': (0.5, 1.0),   
    'gamma': (0.1, 5)
    # 'reg_lambda': (0, 1000, 'log-uniform'),
    # 'reg_alpha': (0, 1.0, 'log-uniform')    
}

def xgboost_hyper_param(learning_rate, n_estimators, max_depth, min_child_weight, subsample, colsample_bytree, gamma):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    clf = XGBRegressor(
        max_depth=max_depth, 
        min_child_weight= min_child_weight,
        learning_rate=learning_rate, 
        n_estimators=n_estimators, 
        subsample=subsample, 
        colsample_bytree=colsample_bytree, 
        gamma=gamma,
        random_state=1,
        eval_metric='logloss'
        # reg_alpha=reg_alpha,
        # reg_lambda=reg_lambda
    )
    return np.mean(cross_val_score(clf, X_train, y_train, cv=5, scoring='r2'))


optimizer_baysein_xgboost = BayesianOptimization( f=xgboost_hyper_param, pbounds=pbounds, random_state=1)
# init_points: 초기 랜덤 포인트 갯수
# acq='ei': Expected Improvement 
# xi=0.01: exploration(불확실성이 가장 높은 점 근처에 최적값이 존재할 것이라는 가정으로 계산된 값) 강도 (보통 0.01) 
optimizer_baysein_xgboost.maximize(init_points=10, n_iter=100, acq='ei', xi=0.01)




|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        | -0.8491   |  0.7085   |  3.63     |  0.01006  |  5.116    |  2.935    |  183.1    |  0.5931   |
|  2        | -2.194    |  0.6728   |  2.044    |  0.274    |  5.934    |  13.7     |  284.0    |  0.9391   |
|  3        | -0.7995   |  0.5137   |  3.385    |  0.2145   |  6.911    |  2.808    |  278.3    |  0.9004   |
|  4        | -13.47    |  0.9841   |  1.636    |  0.3492   |  9.135    |  17.89    |  176.5    |  0.5195   |
|  5        | -1.479    |  0.5849   |  4.403    |  0.05819  |  5.948    |  19.16    |  579.8    |  0.8459   |
|  6        | -2.477    |  0.6578   |  3.464    |  0.419    |  3.128    |  15.0     |  990.0    |  0.8741   |
|  7        | -19.28    |  0.6402   |  3.967    |  0.06058  |  6.135    |  18.17    |  364.3    |  0.6439   |
|  8      

|  45       | -1.092    |  0.7086   |  1.032    |  0.3877   |  9.867    |  6.314    |  237.0    |  0.686    |
|  46       | -0.7038   |  0.8048   |  2.075    |  0.2923   |  5.885    |  0.4002   |  937.2    |  0.8274   |
|  47       | -2.33     |  0.6576   |  0.1488   |  0.23     |  9.006    |  13.01    |  934.2    |  0.7271   |
|  48       | -2.331    |  0.9043   |  3.215    |  0.3923   |  3.976    |  0.1069   |  923.7    |  0.5957   |
|  49       | -1.425    |  0.9554   |  4.442    |  0.1849   |  9.186    |  0.2508   |  949.8    |  0.5515   |
|  50       | -2.018    |  0.9383   |  0.3617   |  0.3638   |  8.496    |  3.457    |  526.1    |  0.583    |
|  51       | -0.8258   |  0.6039   |  4.949    |  0.186    |  9.915    |  2.378    |  260.9    |  0.9528   |
|  52       | -3.766    |  0.5      |  0.1      |  0.5      |  3.0      |  19.39    |  532.6    |  0.5      |
|  53       | -0.3789   |  1.0      |  0.1      |  0.01     |  10.0     |  0.0      |  220.3    |  1.0      |
|  54     

|  91       | -16.61    |  0.5      |  0.1      |  0.01     |  3.0      |  3.229    |  267.6    |  0.5      |
|  92       | -1.053    |  0.6057   |  2.606    |  0.2423   |  8.744    |  6.879    |  220.8    |  0.9479   |
|  93       | -1.043    |  1.0      |  0.1      |  0.5      |  10.0     |  6.353    |  256.6    |  1.0      |
|  94       | -1.147    |  0.5394   |  2.679    |  0.04024  |  9.999    |  5.965    |  993.2    |  0.7189   |
|  95       | -1.183    |  0.918    |  4.435    |  0.1677   |  3.124    |  0.1951   |  254.7    |  0.8643   |
|  96       | -2.105    |  1.0      |  0.1      |  0.5      |  10.0     |  5.086    |  810.6    |  0.5      |
|  97       | -1.539    |  0.5      |  5.0      |  0.5      |  10.0     |  6.249    |  941.2    |  0.5      |
|  98       | -1.089    |  0.7191   |  4.994    |  0.3255   |  8.966    |  7.195    |  295.8    |  0.8      |
|  99       | -0.7564   |  0.7255   |  0.4749   |  0.3397   |  4.069    |  8.518    |  978.8    |  0.9951   |
|  100    

In [424]:
optimizer_baysein_xgboost.max

{'target': -0.07763262940259368,
 'params': {'colsample_bytree': 0.5,
  'gamma': 5.0,
  'learning_rate': 0.01,
  'max_depth': 8.467947985623375,
  'min_child_weight': 0.0,
  'n_estimators': 270.5892332626289,
  'subsample': 1.0}}

In [428]:
mode_baysein_xgboost = XGBRegressor(
 colsample_bytree= 0.5,
  gamma = 5.0,
  learning_rate= 0.01,
  max_depth= int(8.467947985623375),
  min_child_weight= 0.0,
  n_estimators= round(270.5892332626289),
  subsample= 1.0)

In [429]:
mode_baysein_xgboost.fit(X_train,y_train)
mode_baysein_xgboost_predict = mode_baysein_xgboost.predict(X_test)

In [431]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(mode_baysein_xgboost, X_train, y_train))


score = make_scorer(r2_score)

print('r2_score = ', score(mode_baysein_xgboost, X_train, y_train))

mean_squared_log_error =  24.91537349565282
r2_score =  0.9472038767192712


In [430]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(mode_baysein_xgboost, X_test, y_test))


score = make_scorer(r2_score)

print('r2_score = ', score(mode_baysein_xgboost, X_test, y_test))

mean_squared_log_error =  0.8345751184137513
r2_score =  0.05151721471456683


### Bayesian Optimization LightGBM 적용

In [436]:
import numpy as np
from xgboost import XGBRegressor
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor
pbounds = {  
    'learning_rate': (0.01, 0.5),  
    'n_estimators': (100, 1000), 
    'max_depth': (3, 10),
    'min_child_weight': (0, 10),    
    'subsample': (0.5, 1.0),
    'min_samples_leaf' : (1,20),
    'colsample_bytree': (0.5, 1.0),
    'reg_lambda': (0, 1000),
    'reg_alpha': (0, 1.0)
}

def lgbm_hyper_param(learning_rate, n_estimators, max_depth, min_child_weight, subsample, colsample_bytree,min_samples_leaf,reg_lambda,reg_alpha):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    clf = LGBMRegressor(
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        learning_rate=learning_rate, 
        n_estimators=n_estimators, 
        subsample=subsample, 
        colsample_bytree=colsample_bytree,
        random_state=1
        # reg_lambda=reg_lambda,        
        # reg_alpha=reg_alpha
    )
    return np.mean(cross_val_score(clf, X_train, y_train, cv=5, scoring='neg_mean_squared_error'))   # cv 도 숫자로 작성하여, 내부적으로 (Stratified)KFold 사용함

optimizer_baysein_lightgbm = BayesianOptimization( f=lgbm_hyper_param, pbounds=pbounds, verbose=1, random_state=1)
optimizer_baysein_lightgbm.maximize(init_points=10, n_iter=100, acq='ei', xi=0.01)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_sa... | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  2        | -1.716e+2 |  0.7694   |  0.2154   |  7.797    |  2.045    |  17.68    |  124.6    |  0.6705   |  417.3    |  0.7793   |
|  12       | -1.529e+2 |  0.8678   |  0.1266   |  4.345    |  0.9895   |  4.37     |  100.4    |  0.3381   |  661.5    |  0.9486   |
|  21       | -1.425e+2 |  0.7062   |  0.04145  |  3.156    |  8.928    |  4.245    |  160.9    |  0.03734  |  957.5    |  0.6605   |
|  28       | -1.416e+2 |  0.8687   |  0.04259  |  3.496    |  3.538    |  18.78    |  119.0    |  0.5188   |  414.1    |  0.8207   |
|  29       | -1.378e+2 |  0.7361   |  0.01855  |  3.437    |  8.428    |  4.853    |  164.2    |  0.06892  |  956.5    |  0.699    |
|  34       | -1.34e+22 |  1.0      |  0.01     |  4.143    | 

In [437]:
optimizer_baysein_lightgbm.max

{'target': -1.3386271152686441e+22,
 'params': {'colsample_bytree': 1.0,
  'learning_rate': 0.01,
  'max_depth': 3.0,
  'min_child_weight': 1.5265364971550224,
  'min_samples_leaf': 16.74980264800236,
  'n_estimators': 117.07033538647396,
  'reg_alpha': 1.0,
  'reg_lambda': 416.9833751876486,
  'subsample': 1.0}}

In [439]:
params = {'colsample_bytree': round(0.9526337512385443),
  'learning_rate': 0.010247647887474036,
  'max_depth': round(7.89238235933958),
  'min_child_weight': 8.626929257435314,
  'min_samples_leaf': 13.108100454447696,
  'n_estimators': round(446.34210021506914),
  'reg_alpha': 0.2813365885218613,
  'reg_lambda': 332.12650907659867,
  'subsample': 0.7102827318607705}


XGBR = XGBRegressor(colsample_bytree= 1,
  learning_rate= 0.01,
  max_depth =  3,
  min_child_weight= 1.5265364971550224,
  min_samples_leaf= 16.74980264800236,
  n_estimators= round(117.07033538647396),
  reg_alpha= 1.0,
  reg_lambda= 416.9833751876486,
  subsample= 1.0).fit(X_train, y_train)
XGBR_predict = XGBR.predict(X_test)

[16:35:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "min_samples_leaf" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [440]:
score = make_scorer(mean_squared_error)

print('mean_squared_log_error = ', score(XGBR, X_train, y_train))


score = make_scorer(r2_score)

print('r2_score = ', score(XGBR, X_train, y_train))

mean_squared_log_error =  1.623172241991888e+22
r2_score =  -0.09867275490565808


In [385]:
XGBR_predict

array([1.6854599e+10, 2.3762844e+10, 7.5164590e+09, 2.1715263e+10,
       4.1527009e+10, 5.3517980e+10, 4.7246561e+10, 4.9657266e+10,
       5.3039051e+10, 3.6335395e+10, 4.1463636e+10, 4.3321049e+10,
       5.4074991e+10, 5.3881614e+10, 5.3006209e+10, 6.3147790e+10,
       6.3368098e+10, 6.3368098e+10, 5.3355381e+10, 6.3368098e+10,
       6.3368098e+10, 6.3368098e+10, 5.3646447e+10, 5.3112787e+10,
       6.2686499e+10, 6.2193959e+10, 5.2146606e+10, 5.8774327e+10,
       6.2091592e+10, 5.7696088e+10, 6.3368098e+10, 6.3368098e+10,
       6.2061724e+10, 6.2134579e+10, 3.0089038e+10, 5.9419673e+10,
       6.1239525e+10, 6.2223208e+10, 6.2959665e+10, 6.1525987e+10,
       6.0729328e+10, 6.1873725e+10, 5.6168657e+10, 5.9067113e+10,
       6.1516095e+10, 6.1721076e+10, 6.1245092e+10, 1.6184718e+10],
      dtype=float32)

### Bayesian Optimization Random Forest 적용

In [255]:
X_train.isnull().sum()

day       0
month     0
year      0
hour      0
dow       0
woy       0
실적        0
입찰공고차수    0
dtype: int64

In [271]:
# import numpy as np
# from xgboost import XGBRegressor
# from bayes_opt import BayesianOptimization
# from sklearn.model_selection import cross_val_score
# from lightgbm import LGBMRegressor
# pbounds = {"n_estimators": (10, 1000),
#               "max_depth": (1, 150),
#               "min_samples_split": (2, 10),
# }

# def RandomForest_hyper_param(n_estimators, max_depth, min_samples_split):
#     max_depth = max_depth
#     n_estimators = n_estimators
#     clf = RandomForestRegressor(
#         max_depth=max_depth,
#         n_estimators=n_estimators, 
#         min_samples_split=min_samples_split,
#         random_state=1,
#         # reg_lambda=reg_lambda,        
#         # reg_alpha=reg_alpha
#     )
#     return np.mean(cross_val_score(clf, X_train, y_train, cv=5, scoring='r2'))   # cv 도 숫자로 작성하여, 내부적으로 (Stratified)KFold 사용함


# try:
#     optimizer_baysein_randomforest = BayesianOptimization( f=RandomForest_hyper_param, pbounds=pbounds, verbose=1, random_state=1)
#     optimizer_baysein_randomforest.maximize(init_points=10, n_iter=100,acq='ei', xi=0.01)
# except:
#     pass


|   iter    |  target   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------


In [444]:
mode_baysein_xgboost_model = mode_baysein_xgboost

In [445]:
random_model = random_grid_2
extra_model = ExtraTreesRegressor(max_depth= 10, min_samples_leaf= 4, min_samples_split= 4, n_estimators=800)

In [451]:
from sklearn.ensemble import VotingRegressor

Voting = VotingRegressor(estimators = [

        ('XGBoost', XGB_grid_2),
        ('Random Forest', random_model), 
        ('Extra Trees', extra_model),
        ('Baysein_XGboost_model',mode_baysein_xgboost_model)
    
    ],n_jobs=-1)

Voting

VotingRegressor(estimators=[('XGBoost',
                             XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1,
                                          enable_categorical=False, gamma=0,
                                          gpu_id=-1, importance_type=None,
                                          interaction_constraints='',
                                          learning_rate=0.02, max_delta_step=0,
                                          max_depth=4, min_child_weight=1,
                                          min_samples_leaf=12, missing=nan,
                                          monotone_constrain...
                                          gpu_id=-1, importance_type=None,
                                          interaction_constraints='',
                                   

In [452]:
Voting.fit(X_train, y_train)

VotingRegressor(estimators=[('XGBoost',
                             XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1,
                                          enable_categorical=False, gamma=0,
                                          gpu_id=-1, importance_type=None,
                                          interaction_constraints='',
                                          learning_rate=0.02, max_delta_step=0,
                                          max_depth=4, min_child_weight=1,
                                          min_samples_leaf=12, missing=nan,
                                          monotone_constrain...
                                          gpu_id=-1, importance_type=None,
                                          interaction_constraints='',
                                   

In [453]:
voting_pred = grid_hard.predict(X_test)

In [454]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(Voting, X_train, y_train))


score = make_scorer(r2_score)

print('r2_score = ', score(Voting, X_train, y_train))

mean_squared_log_error =  37.725463080548295
r2_score =  0.7793467732452257


In [474]:
score = make_scorer(mean_squared_log_error)

print('mean_squared_log_error = ', score(Voting, X_test, y_test))


score = make_scorer(r2_score)

print('r2_score = ', score(Voting, X_test, y_test))

mean_squared_log_error =  0.7478927526867113
r2_score =  0.08150440835384931


# 이상치를 포함한 채로 분석을 진행하면 너무 performance가 낮게 나온다...!


In [498]:
df_mode_baysein_xgboost_train =  pd.DataFrame(mode_baysein_xgboost.predict(X_train),index=X_train.index)
df_mode_baysein_xgboost_test =  pd.DataFrame(mode_baysein_xgboost.predict(X_test),index=X_test.index)

df_random_grid_extra_2_train =  pd.DataFrame(random_grid_extra_2.predict(X_train),index=X_train.index)
df_random_grid_extra_2_test =  pd.DataFrame(random_grid_extra_2.predict(X_test),index=X_test.index)

df_Voting_train =  pd.DataFrame(mode_baysein_xgboost.predict(X_train),index=X_train.index)
df_Voting_test =  pd.DataFrame(mode_baysein_xgboost.predict(X_test),index=X_test.index)

In [510]:
# Total
fig = go.Figure()

fig.add_trace(go.Scatter(x = df2.index, y = df2[df2_1.columns[0]],name = 'Test_실제값',marker_color = 'black'))
fig.add_trace(go.Scatter(x = df2_1.index, y = df2_1[df2_1.columns[0]],name = 'Train_실제값',marker_color = 'black'))
fig.add_trace(go.Scatter(x = df1.index, y = df1[0], name = 'Test_예측값_XGBOOST',marker_color = 'red'))#,mode='lines+markers'

fig.add_trace(go.Scatter(x = df1_1.index, y = df1_1[0], name = 'Train_예측값_XGBOOST',marker_color = 'red'))


fig.add_trace(go.Scatter(x = df3.index, y = df3[0], name = 'Test_예측값_RandomForest',marker_color = 'green'))

fig.add_trace(go.Scatter(x = df3_1.index, y = df3_1[0], name = 'Train_예측값_RandomForest',marker_color = 'green'))

fig.add_trace(go.Scatter(x = df4.index, y = df4[0], name = 'Test_예측값_LightGB',marker_color = 'blue'))

fig.add_trace(go.Scatter(x = df4_1.index, y = df4_1[0], name = 'Train_예측값_LightGB',marker_color = 'blue'))



fig.add_trace(go.Scatter(x = df_mode_baysein_xgboost_test.index, y = df_mode_baysein_xgboost_test[0], name = 'Test_예측값_Baysein_XGBoost',marker_color = 'purple'))

fig.add_trace(go.Scatter(x = df_mode_baysein_xgboost_train.index, y = df_mode_baysein_xgboost_train[0], name = 'Train_예측값_Baysein_XGBoost',marker_color = 'purple'))



fig.add_trace(go.Scatter(x = df_random_grid_extra_2_test.index, y =df_random_grid_extra_2_test[0], name = 'Test_예측값_Extra',marker_color = 'gray'))

fig.add_trace(go.Scatter(x = df_random_grid_extra_2_train.index, y =df_random_grid_extra_2_train[0], name = 'Train_예측값_Extra',marker_color = 'gray'))



fig.add_trace(go.Scatter(x = df_Voting_test.index, y = df_Voting_test[0], name = 'Test_예측값_Voting',marker_color = 'lightcoral'))

fig.add_trace(go.Scatter(x = df_Voting_train.index, y = df_Voting_train[0], name = 'Train_예측값_Voting',marker_color = 'lightcoral'))



fig.add_vline(x = '2021-01-01',line_dash="dash")
fig.add_annotation(
            x="2021-02-20",
            y=10**12,
            text="<b>Test</b>",
            showarrow=False,
            font=dict(
                size=15,
                color = 'black'),
            align="center",
            ax=40,
            ay=-30,

)
fig.add_annotation(
            x="2020-11-15",
            y=10**12,
            text="<b>Train</b>",
            showarrow=False,
            font=dict(
                size=15,
                color = 'black'
                ),
            align="center",
            ax=40,
            ay=-30,

)

fig.update_layout(
    autosize=False,
    width=1600,
    height=800,
    title= '물품 : 방위사업청 대한 수요예측 '
    )

fig.update_xaxes(title = '주 단위 시계열 (2017/01/05 ~ 2021/01/23)')
fig.update_yaxes(title = '주별 예산 총액')

# fig.write_html("주별.html")
fig.show()

In [501]:
lst_a = []
lst_b = []

score = make_scorer(mean_squared_log_error)

lst_a.append(score(XGB_grid_2, X_train, y_train))
lst_a.append(score(LightGB_grid_2, X_train, y_train))
lst_a.append(score(mode_baysein_xgboost, X_train, y_train))
lst_a.append(score(random_grid_extra_2, X_train, y_train))
lst_a.append(score(random_grid_2, X_train, y_train))
lst_a.append(score(Voting, X_test, y_test))

score = make_scorer(r2_score)

lst_b.append(score(XGB_grid_2, X_train, y_train))
lst_b.append(score(LightGB_grid_2, X_train, y_train))
lst_b.append(score(mode_baysein_xgboost, X_train, y_train))
lst_b.append(score(random_grid_extra_2, X_train, y_train))
lst_b.append(score(random_grid_2, X_train, y_train))
lst_b.append(score(Voting, X_test, y_test))

df_score_train = pd.DataFrame(np.array([lst_a,lst_b]).T,columns = ['mean_squared_log_error','r2_score'],index=['XGBoost','LightGBM','Baysein_XGBoost','Extra_model','Random_Forest','Voting_Regression'])

In [494]:
lst_a = []
lst_b = []

score = make_scorer(mean_squared_log_error)

lst_a.append(score(XGB_grid_2, X_test, y_test))
lst_a.append(score(LightGB_grid_2, X_test, y_test))
lst_a.append(score(mode_baysein_xgboost, X_test, y_test))
lst_a.append(score(random_grid_extra_2, X_test, y_test))
lst_a.append(score(random_grid_2, X_test, y_test))
lst_a.append(score(Voting, X_test, y_test))


score = make_scorer(r2_score)

lst_b.append(score(XGB_grid_2, X_test, y_test))
lst_b.append(score(LightGB_grid_2, X_test, y_test))
lst_b.append(score(mode_baysein_xgboost, X_test, y_test))
lst_b.append(score(random_grid_extra_2, X_test, y_test))
lst_b.append(score(random_grid_2, X_test, y_test))
lst_b.append(score(Voting, X_test, y_test))

df_score_test = pd.DataFrame(np.array([lst_a,lst_b]).T,columns = ['mean_squared_log_error','r2_score'],index=['XGBoost','LightGBM','Baysein_XGBoost','Extra_model','Random_Forest','Voting_Regression'])

In [495]:
df_score_train

,mean_squared_log_error,r2_score
XGBoost,43.057487,0.887947
LightGBM,46.173374,0.581697
Baysein_XGBoost,24.915373,0.947204
Extra_model,31.427397,0.532983
Random_Forest,27.667708,0.531471
Voting_Regression,0.747893,0.081504


In [497]:
df_score_test

,mean_squared_log_error,r2_score
XGBoost,0.737514,0.115466
LightGBM,0.655323,0.098255
Baysein_XGBoost,0.834575,0.051517
Extra_model,0.789319,0.036750
Random_Forest,0.777022,0.080664
Voting_Regression,0.747893,0.081504


In [511]:
df_test_1_grouper

,index,입찰공고차수,합계수량,합계금액,추정가격,배정예산,day,month,year,hour,dow,woy,실적
공고게시일자,,,,,,,,,,,,,
2017-01-29,0,0,5.0,170000000.0,154545455.0,1.708266e+08,24,1,2017,0,1,4,0.873736
2017-02-05,6,0,5.0,170000000.0,154545455.0,1.708266e+08,2,2,2017,0,3,5,0.873736
2017-02-12,7,0,5.0,170000000.0,154545455.0,1.708266e+08,10,2,2017,0,4,6,0.873736
2017-02-19,1,0,1.0,443847600.0,403497818.0,4.454314e+08,16,2,2017,0,3,7,0.873736
2017-02-26,2,0,5.0,170000000.0,154545455.0,1.708266e+08,23,2,2017,0,3,8,0.873736
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-06,12618210,139,1.0,219910800.0,199918909.0,1.158117e+11,875,1262,216140,0,215,5243,0.934931
2020-12-13,10687813,131,0.0,0.0,0.0,7.513172e+10,838,1080,181800,0,208,4500,0.934931
2020-12-20,6544001,82,0.0,0.0,0.0,1.394417e+10,881,660,111100,0,111,2805,0.934931
